This notebook provides code snippets to identify and remove disconnect (and potentially motion) related artifacts in your recordings so that they can be ignored during post-processing.

In [1]:
# Import all the functions you need
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
from pathlib import Path

# Comment here to add in for either mac or linux computer
# sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/NeuroPy'])  # For mac laptop
sys.path.extend(['/data/GitHub/NeuroPy'])  # for linux

from neuropy import core
from neuropy.io import (optitrackio,
                        dlcio,
                        )
from neuropy.io.neuroscopeio import NeuroscopeIO
from neuropy.io.binarysignalio import BinarysignalIO 
from neuropy.io.miniscopeio import MiniscopeIO

plt.rcParams['pdf.fonttype'] = 42


In [4]:
# Define a class for a typical recording or set of recordings
class ProcessData:
    def __init__(self, basepath):
        basepath = Path(basepath)
        self.basepath = basepath
        xml_files = sorted(basepath.glob("*.xml"))
        assert len(xml_files) == 1, "Found more than one .xml file"
        
        fp = xml_files[0].with_suffix("")
        self.filePrefix = fp
        
        self.recinfo = NeuroscopeIO(xml_files[0])
        eegfiles = sorted(basepath.glob('*.eeg'))
        assert len(eegfiles) == 1, "Fewer/more than one .eeg file detected"
        self.eegfile = BinarysignalIO(eegfiles[0], n_channels=self.recinfo.n_channels,
                                     sampling_rate=self.recinfo.eeg_sampling_rate,
                                     )
        try:
            self.datfile = BinarysignalIO(eegfiles[0].with_suffix('.dat'),
                                         n_channels=self.recinfo.n_channels,
                                         sampling_rate=self.recinfo.dat_sampling_rate,
                                         )
        except FileNotFoundError:
            print('No dat file found, not loading')
                
        
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.recinfo.source_file.name})"
    
def Wedge_ani():
    basepath = '/data3/Anisomycin/Recording_Rats/Wedge/2022_12_13_anisomycin50mg/' # for linux desktop
    return ProcessData(basepath)

In [23]:
sess = Wedge_ani()
print(sess.recinfo)

filename: /data3/Anisomycin/Recording_Rats/Wedge/2022_12_13_anisomycin50mg/Wedge_ani1.xml 
# channels: 35
sampling rate: 30000
lfp Srate (downsampled): 1250



In [24]:
print(sess.eegfile)

duration: 18406.83 seconds 
duration: 5.11 hours 



# Preprocess Ephys data
Identify artifacts:  
- disconnects  
- filtering  
  
Send to event files to check in Neuroscope  
Write `dead_times.txt` file for spyking-circus and `artifacts.npy` file for my own analysis.

### Identify disconnect and motion artifacts

In [35]:
from neuropy.analyses.artifact import detect_artifact_epochs
signal = sess.eegfile.get_signal()
art_epochs = detect_artifact_epochs(signal, thresh=7, 
                                    edge_cutoff=4, merge=4)

# art_epochs.source_file = sess.filePrefix.with_suffix(".artifact.npy")
sess.recinfo.write_epochs(epochs=art_epochs, ext='art')

In [36]:
df = art_epochs.to_dataframe()
print(df.duration.sum())
df.sort_values(by=['duration'])

109.73519999998652


,start,stop,label,duration
5,1017.7720,1017.7808,,0.0088
74,12951.8824,12951.9016,,0.0192
6,1030.3752,1030.3944,,0.0192
7,2863.8328,2863.8544,,0.0216
66,10907.5768,10907.6048,,0.0280
...,...,...,...,...
79,13690.0000,13695.9152,,5.9152
85,14802.9776,14809.1096,,6.1320
54,7177.3376,7186.8408,,9.5032
87,14836.6840,14852.5656,,15.8816


In [33]:
# make easy to track in neuroscope
df['start_min'] = np.floor(df.start/60)
df['start_sec'] = (df.start/60 - df['start_min'])*60
df['stop_min'] = np.floor(df.stop/60)
df['stop_sec'] = (df.stop/60 - df['stop_min'])*60

In [37]:
def add_epoch_buffer(epoch_df: pd.DataFrame, buffer_sec: float or int or tuple or list):
    """Extend each epoch by buffer_sec before/after start/stop of each epoch"""
    if type(buffer_sec) in [int, float]:
        buffer_sec = (buffer_sec, buffer_sec)
    else:
        assert len(buffer_sec) == 2
        
    epoch_df['start'] -= buffer_sec[0]
    epoch_df['stop'] += buffer_sec[1]

In [38]:
add_epoch_buffer(art_epochs._epochs, 0.4)
sess.recinfo.write_epochs(art_epochs, 'art')
art_epochs._epochs

,start,stop,label
0,20.0664,21.0344,
1,30.0928,30.9800,
2,97.5776,100.8160,
3,193.6952,194.7064,
4,715.9752,716.9192,
...,...,...,...
96,17962.9080,17963.8952,
97,17968.3616,17971.7952,
98,18038.2016,18039.2040,
99,18076.4520,18077.4552,


In [39]:
art_epochs.save(sess.filePrefix.with_suffix('.art_epochs.npy'))

/data3/Anisomycin/Recording_Rats/Wedge/2022_12_13_anisomycin50mg/Wedge_ani1.art_epochs.npy saved


In [74]:
from neuropy.io.spykingcircusio import SpykingCircusIO
from neuropy.core import Epoch
combined_epochs = Epoch(epochs=combine_epochs(art_epochs_combined, inplace=False))

SpykingCircusIO.write_epochs(sess.basepath / 'dead_times.txt', 
                            combined_epochs)
sess.recinfo.write_epochs(combined_epochs, 'art')

dead_times.txt created


# Optional code below to combine other epochs with above

In [41]:
def combine_epochs(epochs_df: pd.DataFrame, inplace: bool = True):
    """Combine epochs so that there are no starts or stops contained
    entirely within another epoch"""
    
    all([col in epochs_df.columns for col in ["start", "stop"]])
   
    # First find epochs that overlap and get id to replace with
    start_overlaps, stop_overlaps = [], []
    for ide, epoch in epochs_df.iterrows():
        overlap_start = np.bitwise_and(epoch['start'] > epochs_df['start'],
                                       epoch['start'] < epochs_df['stop'])
        overlap_stop = np.bitwise_and(epoch['stop'] > epochs_df['start'],
                                       epoch['stop'] < epochs_df['stop'])
        if overlap_start.sum() == 1:
            start_overlap_id = np.where(overlap_start)[0][0]
#             print('epoch ' + str(ide) + ' overlap start w epoch ' 
#                   + str(start_overlap_id))
            start_overlaps.append([ide, start_overlap_id])

        if overlap_stop.sum() == 1:
            stop_overlap_id = np.where(overlap_stop)[0][0]
#             print('epoch ' + str(ide) + ' overlap stop w epoch '
#                  + str(stop_overlap_id))
            stop_overlaps.append([ide, stop_overlap_id])
    # Now walk through and replace
    for start in start_overlaps:
        epochs_df.loc[start[0], "start"] = epochs_df.loc[start[1], "start"]

    for stop in stop_overlaps:
        epochs_df.loc[stop[0], "stop"] = epochs_df.loc[stop[1], "stop"]

#     overlap_ids = np.hstack((np.asarray(start_overlaps)[:, 1],
#                   np.asarray(stop_overlaps)[:, 1]))
    
#     print('Dropping duplicate epochs: ' + str(overlap_ids))
    
    if inplace:
        epochs_df.drop_duplicates(inplace=inplace, ignore_index=True)
        epochs_df.sort_values(by='start', inplace=inplace, ignore_index=True)
    
        return None
    else:       
        epochs_out = epochs_df.drop_duplicates(inplace=inplace, ignore_index=True)
        epochs_out = epochs_out.sort_values(by='start', inplace=inplace, ignore_index=True)

        return epochs_out

In [73]:
# Add in loosening of headstage not caught well by automatic artifact detection
art_by_hand = Epoch(pd.DataFrame({"start": [246*60 + 31.1], "stop": [247*60 + 32.766], "label": "by_hand"}))

art_epochs_combined = pd.concat((art_epochs._epochs, art_by_hand._epochs), 
                                keys=["start", "stop", "label"], ignore_index=True)
art_epochs_combined = art_epochs_combined.sort_values(by='start', ignore_index=True)


Test code to verify that merging of epochs works properly

In [ ]:
# unit test #1 - one epoch entirely within another
# epochs_test = art_epochs_combined.head(4).copy()
epochs_test = pd.DataFrame({"start": [0, 1079.28, 1130.82, 1131.13],
                            "stop": [0.2, 1082.91, 1140.80, 1131.53],
                            "label": ""})
print(epochs_test)
epochs_test = combine_epochs(epochs_test, inplace=False)
print(epochs_test)

#unit test #2 - start of one epoch in a different epoch.
epochs_test = pd.DataFrame({"start": [0, 1079.28, 1130.82, 1131.13],
                            "stop": [0.2, 1082.91, 1140.80, 1141.53],
                            "label": ""})
print(epochs_test)
combine_epochs(epochs_test)
print(epochs_test)
epochs_test = pd.DataFrame({"start": [0, 1079.28, 1130.82, 1130.13],
                            "stop": [0.2, 1082.91, 1140.80, 1131.53],
                            "label": ""})
print(epochs_test)
epochs_test = combine_epochs(epochs_test, inplace=False)
print(epochs_test)
#unit test #3 - stop of one epoch in a different epoch